# HORTON 3.x DensPart Demo

This notebook shows how to use a `.fchk` file to run an minimal basis iterative stockholder (MBIS) partitioning using the [`IOData`](https://github.com/theochem/iodata), [`Grid`](https://github.com/theochem/grid), [`GBasis`](https://github.com/theochem/gbasis), and [`DensPart`](https://github.com/theochem/denspart) packages.

> **If you are using Google Colab**, you'll need to install packages and download the example files by running the commands in the next cell.

In [1]:
# Install packages in Google Colab. Don't run this cell if packages/data is already in your environment
# Manually specifying the scipy version because of current gbasis issues.
! pip install scipy==1.10.1
! pip install git+https://github.com/theochem/iodata.git
! pip install git+https://github.com/theochem/grid.git
! pip install git+https://github.com/theochem/gbasis.git
! pip install git+https://github.com/theochem/denspart.git

# download the example files.
# If you are using your own files, you'll set up the path to the file here.
import os
from urllib.request import urlretrieve

fpath = "data/"
if not os.path.exists(fpath):
    os.makedirs(fpath, exist_ok=True)

urlretrieve(
    "https://raw.githubusercontent.com/theochem/horton3/master/notebooks/data/h2o_sto3g.fchk",
    os.path.join(fpath, "h2o_sto3g.fchk")
    )

  Cloning https://github.com/theochem/iodata.git to /tmp/pip-req-build-wlsash1x
  Running command git clone --filter=blob:none --quiet https://github.com/theochem/iodata.git /tmp/pip-req-build-wlsash1x
  Resolved https://github.com/theochem/iodata.git to commit ca6113b60fc967f3c280bd03374aaba1644d672e
  Preparing metadata (setup.py) ... done
  Created wheel for qc-iodata: filename=qc_iodata-0.0.0.post0-py3-none-any.whl size=3337356 sha256=bf7773ceea26b3cd12cf4c05d14a64458aa7744cb98f4d0e62751052543a973e
  Stored in directory: /tmp/pip-ephem-wheel-cache-ygg9oswj/wheels/c2/0a/62/493efe62d77a51ee3d7918ade8cfe73f1d18416f9679edd471
Successfully built qc-iodata
  Cloning https://github.com/theochem/grid.git to /tmp/pip-req-build-ghecgf00
  Running command git clone --filter=blob:none --quiet https://github.com/theochem/grid.git /tmp/pip-req-build-ghecgf00
  Resolved https://github.com/theochem/grid.git to commit af94e5bee1868d1533c30c164c41f0dbe9dd470f
  Installing build dependencies ... done

('data/h2o_sto3g.fchk', <http.client.HTTPMessage at 0x7c8fc2bd1420>)

# `IOData`

[GitHub Repository](https://github.com/theochem/iodata) | [Webpage](https://iodata.qcdevs.org/) | [Publication](https://onlinelibrary.wiley.com/doi/10.1002/jcc.26468)

Check [IOData Webpage](https://iodata.qcdevs.org/) for list of supported file formats and properties.

Here we use `IOData` to load an `.fchk` file.

In [2]:
# This uses IOData to load an *.fchk file
from iodata import load_one

mol = load_one("data/h2o_sto3g.fchk")

# `Grid`

[GitHub Repository](https://github.com/theochem/grid) | [Webpage](https://grid.qcdevs.org/)

Here we make an "ultrafine" Becke-Lebedev molecular grid. Other types of grids can be constructed too.

In [3]:
# Make Becke-Lebedev molecular grid (using preset grid)
from grid.becke import BeckeWeights
from grid.molgrid import MolGrid
from grid.onedgrid import GaussChebyshev
from grid.rtransform import BeckeRTransform

oned = GaussChebyshev(100)
rgrid = BeckeRTransform(1e-4, 1.5).transform_1d_grid(oned)
grid = MolGrid.from_preset(mol.atnums, mol.atcoords, rgrid, "ultrafine", BeckeWeights())

/usr/local/lib/python3.10/dist-packages/grid/atomgrid.py:889: UserWarning: Lebedev weights are negative which can introduce round-off errors.
  sphere_grid = AngularGrid(degree=deg_i, use_spherical=use_spherical)


# `GBasis`

[GitHub Repository](https://github.com/theochem/gbasis)

Compute molecular density (using gbasis package). This is done by constructing the density matrix in the atomic-orbital basis,

$$
\gamma_{pr} = \sum_{q} c_{pq} n_q c_{qr}
$$

and then evaluating the electron density on the grid.


In [4]:
from gbasis.wrappers import from_iodata
from gbasis.evals.density import evaluate_density

import numpy as np   #We need this to evaluate the 1DM from the MOs

one_rdm = np.dot(mol.mo.coeffs * mol.mo.occs, mol.mo.coeffs.T)
basis = from_iodata(mol)
density = evaluate_density(one_rdm, basis[0], grid.points, coord_type=basis[1])

# `Denspart`

Compute and print atomic charges and atomic dipoles/multipoles

For details about variational Hirshfeld partitioning and the MBIS method, see:
- [Information-Theoretic Atoms in Molecules](https://doi.org/10.1021/acs.jpca.7b08966).
- [Minimal Basis Iterative Stockholder (MBIS)](https://pubs.acs.org/doi/abs/10.1021/acs.jctc.6b00456).

In [5]:
from denspart.mbis import MBISProModel
from denspart.vh import optimize_reduce_pro_model
from denspart.properties import compute_radial_moments, compute_multipole_moments, safe_ratio

pro_model_init = MBISProModel.from_geometry(mol.atnums, mol.atcoords)
pro_model, localgrids = optimize_reduce_pro_model(pro_model_init,grid,density)

results = pro_model.to_dict()
results.update({"charges": pro_model.charges,
                 "multipole_moments": compute_multipole_moments(pro_model, grid, density, localgrids)})

print("Atomic Charges    = ", results["charges"])
print("")
print("Atomic Multipoles = ", results["multipole_moments"])

Building local grids
Integral of density: 10.000000007189016
Optimization
#Iter  #Call         ekld          kld  -constraint     grad.rms  cputime (s)
-----  -----  -----------  -----------  -----------  -----------  -----------
    1      1    0.5936643    0.5936643  -7.1890e-09   2.4770e-01    0.0557489
    2      2    0.4224869    0.4067274   1.5760e-02   1.7380e-01    0.0508478
    3      3    2.7972750    2.2696992   5.2758e-01   1.6102e+00    0.0531064
    4      4    0.6307481    0.0908026   5.3995e-01   2.1510e-01    0.0519477
    5      5    1.7407048    2.7852702  -1.0446e+00   1.2625e+00    0.0581814
    6      6    0.2765421   -0.1087255   3.8527e-01   8.2597e-02    0.0525186
    7      6    0.2765421   -0.1087255   3.8527e-01   8.2597e-02    0.0525186
    8      7    0.2560120    0.0162623   2.3975e-01   5.2435e-02    0.0436702
    9      8    0.2496596    0.0234474   2.2621e-01   4.9950e-02    0.0343531
   10      9    0.2334682    0.4039918  -1.7052e-01   3.1926e-02    

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


   22     23    0.2295481    0.2627020  -3.3154e-02   5.9980e-03    0.0421575
   23     24    0.2291385    0.2577497  -2.8611e-02   5.3032e-03    0.0351705
   24     25    0.2279101    0.2039434   2.3967e-02   6.0058e-03    0.0314418
   25     26    0.2278231    0.1973589   3.0464e-02   6.3806e-03    0.0438370
   26     27    0.2267634    0.1962649   3.0499e-02   8.9342e-03    0.0314270
   27     28    0.2265583    0.2144687   1.2090e-02   5.5002e-03    0.0437023
   28     29    0.2253243    0.2194405   5.8838e-03   1.0711e-02    0.0377451
   29     31    6.7464375    7.0411862  -2.9475e-01   3.7579e+00    0.0386752
   30     32    0.2292962    0.1938794   3.5417e-02   2.7527e-02    0.0311889
   31     33    0.2249490    0.2070437   1.7905e-02   9.9310e-03    0.0400437
   32     34    0.2245992    0.2277165  -3.1172e-03   3.6622e-03    0.0350436
   33     35    0.2570640   -0.0351203   2.9218e-01   1.3197e-01    0.0315852
   34     36    0.2245693    0.2112767   1.3293e-02   3.7442e-03